In [1]:
from avian_vocalizations import data
from sklearn.metrics import accuracy_score
from avian_vocalizations.data import AudioFeatureGenerator
from avian_vocalizations.model import ModelFactory
from sklearn.preprocessing import LabelEncoder
import os
from glob import glob
import numpy as np
import warnings; warnings.simplefilter('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

data_dir = '../data'
index_df, shapes_df, train_df, test_df = data.load_data(data_dir)

label_encoder = LabelEncoder().fit(index_df['english_cname'] )

n_classes = len(label_encoder.classes_)
print("The dataset contains %i distinct species labels."%n_classes)

Using TensorFlow backend.


The dataset contains 91 distinct species labels.


In [2]:
cnn = ModelFactory(n_classes)
cnn.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 64)      640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 42, 42, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 42, 42, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 40, 40, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 13, 13, 64)        0         
______________________________________

In [3]:
cnn.load_weights(os.path.join(data_dir,"weights.best.hdf5"))

In [4]:
X_test = index_df.loc[index_df['test']==True].index.values
y_test = label_encoder.transform(index_df.loc[index_df['test']==True,"english_cname"].values)

test_generator = AudioFeatureGenerator(X_test, y_test, batch_size=len(X_test), data_dir=data_dir,
                                       seed=37, n_frames=128, n_classes=n_classes)

Xs, ys = test_generator[0] # batch_size=len(X_test), so one batch has all test samples. 

predictions = cnn.predict(Xs) 
test_accuracy = accuracy_score(np.argmax(ys,axis=1), np.argmax(predictions, axis=1))
print("Test accuracy of benchmark model: %.5f"%test_accuracy)


Test accuracy of benchmark model: 0.24835


## Stability testing

Changing the `seed` value passed to `AudioFeatureGenerator` will change the crop windows for each sample. Let's see how accuracy varies with successive test rounds in order to assess model stability.  

In [10]:
for i in range(3):
    test_generator = AudioFeatureGenerator(X_test, y_test, batch_size=len(X_test), data_dir=data_dir,
                                       seed=37+i, n_frames=128, n_classes=n_classes)
    Xs, ys = test_generator[0] 

    predictions = cnn.predict(Xs) 
    test_accuracy = accuracy_score(np.argmax(ys,axis=1), np.argmax(predictions, axis=1))
    print("Test accuracy of benchmark model: %.5f"%test_accuracy)

Test accuracy of benchmark model: 0.24066
Test accuracy of benchmark model: 0.22418
Test accuracy of benchmark model: 0.22198


## Module Versions

In [7]:
from avian_vocalizations.toolbench import print_module_versions
print_module_versions(globals())

             IPython  {'7.9.0'}
 avian_vocalizations  {'0.1.0'}
             sklearn  {'0.21.3'}
               numpy  {'1.17.3'}
              pandas  {'0.25.2'}
               keras  {'2.3.1'}
